#### Dependencies

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import time
import multiprocessing as mp
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import sys
from contextlib import closing
import psutil
import tracemalloc
import threading
import gc

from essentia.standard import (
    MonoLoader,
    Danceability,
    Spectrum,
    FrameCutter,
    Loudness,
    RhythmExtractor2013,
    KeyExtractor,
    Energy,
    TonalExtractor,
    Inharmonicity,
    MFCC,
    OnsetRate,
    SpectralCentroidTime,
    DynamicComplexity,
    SpectralPeaks,
    NoveltyCurve,
    Spectrum,
    FrameGenerator,
    Windowing,
    MelBands,
    BeatsLoudness,
    Beatogram,
    Meter,
)

2024-11-01 11:46:41.481956: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Global Constants

In [2]:
load_dotenv()
DOWNLOAD_FOLDER = os.getenv('DOWNLOAD_FOLDER')
CPU_THREADS = int(os.getenv('CPU_THREADS'))

#### Data

In [3]:
songs_data = pd.read_csv('data/songs_final.csv')

#### Feature Extraction Functions

In [4]:
def create_spectrogram_image(spectrogram_db, sample_rate):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spectrogram_db, sr=sample_rate, x_axis='time', y_axis='mel', fmax=11025)
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"Mel-Spectrogram")
    plt.tight_layout()
    plt.show()
    plt.close()

In [5]:
def mp3_to_spectrogram(audio_path, sample_rate, create_image=False):
    mp3, _ = librosa.load(audio_path, sr=sample_rate)
    spectrogram = librosa.feature.melspectrogram(y=mp3, sr=sample_rate, n_mels=128, fmax=11025)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    if create_image:
        create_spectrogram_image(spectrogram_db, sample_rate)

    return spectrogram_db

In [6]:
def get_mel_bands(audio):
    spectrum = Spectrum()
    frame_generator = FrameGenerator(audio, frameSize=2048, hopSize=1024)
    window = Windowing(type='hann')

    mel_bands = MelBands(numberBands=40)
    mel_band_energies = []

    for frame in frame_generator:
        spec = spectrum(window(frame))
        mel_band_energies.append(mel_bands(spec))

    mel_band_energies = np.array(mel_band_energies)

    return mel_band_energies

In [ ]:
def run_essentia_algorithms(audio44k, audio16k):
    _, mfcc_coeffs = MFCC(inputSize=len(audio16k))(audio16k)
    danceability_score = Danceability()(audio44k)
    loudness_score = Loudness()(audio16k)
    bpm, beat_positions, _, _, _ = RhythmExtractor2013(method="multifeature")(audio44k)
    key, scale, _ = KeyExtractor()(audio44k)
    energy_score = Energy()(audio16k)

    ### Chord Significances
    _, _, _, _, chords, _, _, _, _, _, _, _ = TonalExtractor()(audio44k)
    unique_chords, counts = np.unique(chords, return_counts=True)
    chords_significance = {chord: significance for (chord, significance) in zip(unique_chords, counts)}

    ### Inharmonicity
    frames = []
    frameCutter = FrameCutter()
    while True:
        frame = frameCutter(audio44k)
        if not len(frame):
            break
        frames.append(frame)
        
    spectrum_magnitudes = []
    for frame in frames:
        spectrum_magnitudes_frame = Spectrum()(frame)
        spectrum_magnitudes.append(spectrum_magnitudes_frame)
    spectrum_magnitudes = np.array(spectrum_magnitudes).flatten()
    
    frequencies, magnitudes = SpectralPeaks()(audio44k)
    hnr_score = None
    if len(frequencies) > 0 and frequencies[0]: 
        hnr_score = Inharmonicity()(frequencies, magnitudes)
    ###
    
    onset_rate_score = OnsetRate()(audio44k)
    brightness_score = SpectralCentroidTime()(audio44k)
    dynamic_complexity_score, _ = DynamicComplexity()(audio16k)
    
    mel_bands = get_mel_bands(audio44k)
    novelty_curve = NoveltyCurve()(mel_bands)
    novelty_score = np.median(np.abs(np.diff(novelty_curve)))
    
    beats_loudness, beats_loudness_band_ratio = BeatsLoudness(beats=beat_positions)(audio44k)
    beatogram = Beatogram()(beats_loudness, beats_loudness_band_ratio)
    time_signature = Meter()(beatogram)

    features = {
        'Danceability': danceability_score[0],
        'Loudness': loudness_score,
        'BPM': bpm,
        'Key': key,
        'Key Scale': scale,
        'Energy': energy_score,
        'Chords Significance': chords_significance,
        'Inharmonicity': hnr_score,
        'Timbre': np.mean(mfcc_coeffs),
        'Onset Rate': onset_rate_score[1],
        'Brightness': brightness_score,
        'Dynamic Complexity': dynamic_complexity_score,
        'Novelty': novelty_score,
        'Time Signature': time_signature,
    }

    return features

In [8]:
def extract_audio_features(audio_file):
    # Load the audio file
    audio44k = MonoLoader(filename=audio_file)()
    audio16k = MonoLoader(filename=audio_file, sampleRate=16000)()

    # Run algorithms
    algorithm_features = run_essentia_algorithms(audio44k, audio16k)

    # Merge results
    return algorithm_features

#### Util Functions

In [9]:
def get_total_memory_usage(process):
    memory_summary = {f'Process {process.pid}': process.memory_info().rss / (1024 * 1024)}
    for child in process.children(recursive=True):
        memory_summary = memory_summary | {f'Child Process {child.pid}': child.memory_info().rss / (1024 * 1024)}
    return memory_summary

def print_memory_usage(process):
    print(get_total_memory_usage(process))
    snapshot = tracemalloc.take_snapshot()
    print(f"Top Consumer of Process {process.pid}: {snapshot.statistics('lineno')[0]}")

def monitor_memory_usage(process, kill_thread, interval=120):
    while True:
        try:
            if kill_thread.value:
                print("MONITOR THREAD KILLED")
                return
            print_memory_usage(process)
        except Exception as e:
            print(f"Thread ERROR: {e}")
            return
        time.sleep(interval)

#### Main Code

In [10]:
# Class constructed from song path
# Song path must follow this format: /some/path/(int)^(video id)^(title).mp3
#                               e.g  /some/path/0^LlWGt_84jpg^Special Breed.mp3
class SongPath:
    def __init__(self, song_path: str):
        self.path = song_path
        self.filename = os.path.basename(song_path)

        song_filename_split = self.filename.split('^')
        if len(song_filename_split) != 3:
            raise Exception("The song's filename doesn't follow the correct format: /some/path/(int)^(video id)^(title).mp3")
        
        self.index, self.video_id, self.title_with_extension = song_filename_split

        self.index = int(self.index)
        self.title = os.path.splitext(self.title_with_extension)[0]

    def __str__(self):
        return f"Idx: {self.index},  videoID: {self.video_id}, title: {self.title_with_extension}"

In [11]:
def process_song(args):
    song_path, output_file = args
    song = SongPath(song_path)
    song_features = extract_audio_features(song.path)
    #print_memory_usage(psutil.Process(os.getpid()))
    #song_results.append((song.index, song_features))

    # If file doesnt exist, create file and add headers
    print(song_features.keys())
    if not os.path.exists(output_file):
        empty_df = pd.DataFrame(columns=song_features.keys())
        empty_df.to_csv(output_file, index=False)
    with open(output_file, 'a') as f:
        pd.DataFrame([song_features]).to_csv(f, header=False, index=False)

def process_song(args):
    song_path, song_results = args
    song = SongPath(song_path)
    song_features = extract_audio_features(song.path)
    #print_memory_usage(psutil.Process(os.getpid()))
    song_results.append((song.index, song_features))

In [12]:
def process_songs():                   
    tracemalloc.start()

    # Get downloaded songs paths
    song_paths = np.array([os.path.join(DOWNLOAD_FOLDER, song_filename) for song_filename in os.listdir(DOWNLOAD_FOLDER)])
    #songs_data_lower, songs_data_higher = [0, len(song_paths)//6]
    songs_data_lower, songs_data_higher = [0, 12]
    song_paths = song_paths[songs_data_lower:songs_data_higher]
    
    # Monitor processes' memory for debugging
    main_process = psutil.Process(os.getpid())
    print(f"Main Process ID: {main_process.pid}")
    memory_thread = threading.Thread(target=monitor_memory_usage, args=(main_process))
    memory_thread.daemon = True  # Ensure the thread will exit when the main program exits
    memory_thread.start()

    # Determine output file (needs to be a brand new file)
    output_file_name, output_file_ext = os.path.splitext(FEATURES_OUTPUT_FILE)
    output_file_suffix = 1
    while os.path.exists(output_file_name + str(output_file_suffix) + output_file_ext):
        output_file_suffix += 1
    output_file = output_file_name + str(output_file_suffix) + output_file_ext

    with mp.Pool(processes=CPU_THREADS, maxtasksperchild=5) as pool:
        with tqdm(total=len(song_paths), desc="Processing songs") as pbar:
            args = [(song_path, output_file) for song_path in song_paths]
            for _ in pool.imap(process_song, args, chunksize=1):
                pbar.update(1)

In [13]:
def process_songs():                   
    tracemalloc.start()

    song_paths = np.array([os.path.join(DOWNLOAD_FOLDER, song_filename) for song_filename in os.listdir(DOWNLOAD_FOLDER)])

    songs_data_lower, songs_data_higher = [len(song_paths)//6*2, len(song_paths)//6*3]
    song_paths = song_paths[songs_data_lower:songs_data_higher]
    
    with mp.Manager() as manager:
        kill_thread = manager.Value('b', False)
        shared_song_results = manager.list()
        main_process = psutil.Process(os.getpid())
        print(f"Main Process ID: {main_process.pid}")
        memory_thread = threading.Thread(target=monitor_memory_usage, args=(main_process, kill_thread))
        memory_thread.daemon = True  # Ensure the thread will exit when the main program exits
        memory_thread.start()
        
        with mp.Pool(processes=CPU_THREADS, maxtasksperchild=10) as pool:
            with tqdm(total=len(song_paths), desc="Processing songs") as pbar:
                args = [(song_path, shared_song_results) for song_path in song_paths]
                for _ in pool.imap(process_song, args, chunksize=1):
                    pbar.update(1)

        kill_thread.value = True
        song_results = list(shared_song_results)

    # Aggregate results in the pandas dataframe
    songs_data_full = songs_data.copy(deep=True)
    for song_index, song_features in song_results:
        for feature, value in song_features.items():
            if feature not in songs_data_full.columns and isinstance(value, (tuple, set, list, np.ndarray, dict)):
                songs_data_full[feature] = np.nan
                songs_data_full[feature] = songs_data_full[feature].astype(object)
            songs_data_full.at[song_index, feature] = value

    return songs_data_full

In [ ]:
songs_data_full = process_songs()

Main Process ID: 477731


Processing songs:   0%|          | 0/16391 [00:00<?, ?it/s]

{'Process 477731': 449.30078125, 'Child Process 477968': 328.55859375, 'Child Process 477978': 343.9453125}
Top Consumer of Process 477731: /tmp/ipykernel_477731/251487092.py:4: size=55.1 MiB, count=3, average=18.4 MiB
{'Process 477731': 462.66796875, 'Child Process 477968': 328.98046875, 'Child Process 477978': 612.13671875, 'Child Process 477981': 601.47265625, 'Child Process 477984': 613.86328125, 'Child Process 477987': 640.109375, 'Child Process 477990': 603.421875, 'Child Process 477993': 677.4609375, 'Child Process 477996': 595.828125, 'Child Process 477999': 627.2265625}
Top Consumer of Process 477731: /tmp/ipykernel_477731/251487092.py:4: size=55.1 MiB, count=3, average=18.4 MiB
{'Process 477731': 464.05078125, 'Child Process 477968': 330.5078125, 'Child Process 478659': 817.9375, 'Child Process 478672': 640.47265625, 'Child Process 478733': 595.1171875, 'Child Process 478763': 611.8046875, 'Child Process 478795': 802.8046875, 'Child Process 478828': 718.12109375, 'Child Proce

IndexError: index 0 is out of bounds for axis 0 with size 0

Thread ERROR: [Errno 32] Broken pipe


In [15]:
songs_data_full.to_csv('data/songs_data_full_4.csv')

NameError: name 'songs_data_full' is not defined

In [ ]:
songs_data_full.dropna(subset=['Danceability'])


,title,artist,views,videoID,duration,Danceability,Loudness,BPM,Key,Key Scale,Energy,Chords Significance,Inharmonicity,Timbre,Onset Rate,Brightness,Dynamic Complexity,Novelty,Time Signature
2,Time Dawdles Immersion,Happy Moppy Puppy,19,LlyA3VwQGfk,47,0.885005,250.051102,82.790154,C#,major,3794.436768,"{'Ab': 25, 'Abm': 36, 'B': 254, 'Bbm': 20, 'Bm...",0.264638,-46.799545,0.816523,448.618805,8.755107,0.120020,10.0
24,Olfactory,Willow Feyth,7,0LEg-Xv1jLs,177,1.127948,43.348568,89.510422,C,minor,277.489044,"{'Ab': 16, 'Abm': 21, 'Am': 169, 'B': 109, 'Bb...",0.210938,-45.489132,4.022096,1465.520630,9.326339,0.000029,38.0
25,Laconicum Cooked,stockydiss,7,-hJengtTa1Q,122,1.240165,2397.812988,137.756180,C#,major,110789.937500,"{'A': 137, 'Ab': 554, 'Abm': 190, 'Am': 98, 'B...",0.440899,-9.262972,3.662256,452.442719,3.564298,120.175781,2.0
26,SayPlease.,Matthew Jabez,5,prKQLzrQTm4,297,1.216394,7224.878418,96.983192,B,minor,574709.562500,"{'A': 1692, 'Am': 278, 'B': 22, 'Bb': 22, 'Bbm...",0.390335,-8.763083,4.730560,377.538361,4.425782,58.746582,16.0
27,Nondirectional (feat. Nissim),Jim Lujan,15,2iA0Nv2Zc_o,157,1.044808,447.419495,97.339424,Bb,minor,9042.547852,"{'A': 366, 'Ab': 11, 'Abm': 15, 'Am': 133, 'B'...",0.239013,-30.008791,2.660975,1975.559448,4.928504,0.514495,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39113,Unpeopled Space,Daniel Rossen,2594,Oih7105oV_I,369,1.056749,4496.832031,134.584778,G,minor,283204.000000,"{'A': 75, 'Ab': 459, 'Am': 193, 'Bb': 181, 'Bb...",NaN,-17.903786,4.127495,1034.967529,4.747279,254.483887,4.0
39114,Petunias,SuperGiant,1462,N30vk7PvGT0,287,1.229671,3673.460938,91.434509,C,minor,209415.078125,"{'A': 39, 'Ab': 195, 'Am': 87, 'B': 91, 'Bb': ...",NaN,-17.526287,3.452802,1822.581177,2.328594,368.899902,4.0
39115,Perfect,"Punctual, Lewis Thompson",2938,b2dcZZyII94,139,1.315047,3198.454346,128.058350,Ab,major,170315.359375,"{'Ab': 522, 'Abm': 231, 'B': 241, 'Bb': 160, '...",0.287090,-17.732653,5.462124,2404.071777,3.657988,10651.343750,2.0
39117,The Wild Colonial Boy,Tommy Makem,6893,nK3t6xlibdc,194,1.035069,1881.050049,114.470016,C,major,77119.570312,"{'Am': 87, 'B': 34, 'Bb': 15, 'Bm': 115, 'C': ...",0.047506,-20.972248,3.817227,1507.105347,2.419128,383.720947,16.0
